In [166]:
import pandas as pd

In [167]:
import numpy as np


    header = None — загрузить без строки с заголовком;
    skiprows=n — пропустить n строк (часто у документов бывает техническая «шапка»);
    encoding — загрузить в конкретной кодировке;
    na_values — список значений, который нужно заменить на NaN (специальный объект, обозначающий пропущенное значение).


In [194]:
log = pd.read_csv("data/log.csv", header = None)

In [ ]:
users = pd.read_csv("data/users.csv", encoding="KOI8-R", sep="	")

In [169]:
log.columns = ['user_id','time','bet','win']

In [ ]:
users.columns = ['user_id','email','geo']

In [195]:
log.head()

,0,1,2,3
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


In [ ]:
log_win = log[log['win'] > 0]

In [ ]:
win_count = log_win.shape[0]
print(win_count)

In [ ]:
log2 = log.query("bet<2000 & win>0")

In [ ]:
log2

In [ ]:
log.user_id.unique()

In [171]:
log = log[~log.user_id.str.contains("error", na=False)]

In [172]:
def get_user_id(s):
    pos = s.find("user")
    if pos > -1:
        return s[pos:]
    else:
        return ""

In [173]:
log['user_id'] = log.user_id.apply(get_user_id)


In [174]:
#t = log.time[0]
log['time'] = log[log.time.str.match("\[", na=False)]['time'].apply(lambda t: str(t).replace('[', ''))
#t = t[1:]

In [175]:
log['time'] = pd.to_datetime(log['time'])

In [ ]:
np.isnan(log.time.values).sum()

In [176]:
log_backup = log.copy()

In [ ]:
log = log_backup.copy()

In [ ]:
log.isnull()

In [190]:
log.dropna(axis=0, subset=['user_id', 'time'])

,user_id,time,bet,win,daytime
0,user_919,2019-01-01 14:06:51,NaN,NaN,day
1,user_973,2019-01-01 14:51:16,NaN,NaN,day
2,user_903,2019-01-01 16:31:16,NaN,NaN,day
3,user_954,2019-01-01 17:17:51,NaN,NaN,day
4,user_954,2019-01-01 21:31:18,NaN,NaN,elvin
...,...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0,day
992,user_967,2019-04-20 14:59:36,10154.0,NaN,day
993,user_973,2019-04-20 17:09:56,10254.0,NaN,day
994,user_977,2019-04-20 18:10:07,10354.0,NaN,elvin


In [ ]:
log = log.drop_duplicates(subset=['user_id', 'time'])

In [ ]:
log.info()

In [ ]:
log.time.max()

In [193]:
log

,user_id,time,bet,win,daytime,hour
0,user_919,2019-01-01 14:06:51,NaN,NaN,day,14
1,user_973,2019-01-01 14:51:16,NaN,NaN,day,14
2,user_903,2019-01-01 16:31:16,NaN,NaN,day,16
3,user_954,2019-01-01 17:17:51,NaN,NaN,day,17
4,user_954,2019-01-01 21:31:18,NaN,NaN,elvin,21
...,...,...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0,day,12
992,user_967,2019-04-20 14:59:36,10154.0,NaN,day,14
993,user_973,2019-04-20 17:09:56,10254.0,NaN,day,17
994,user_977,2019-04-20 18:10:07,10354.0,NaN,elvin,18


In [187]:
def hour_to_daytime(hours):
    if hours >=0 and hours < 6:
        return 'night'
    elif hours >= 6 and hours < 12:
        return 'morning'
    elif hours >= 12 and hours < 18:
        return 'day'
    else:
        return 'elvin'

In [188]:
log['daytime'] = log.time.dt.hour.apply(hour_to_daytime)

In [189]:
log.daytime.value_counts(ascending=True)

elvin      227
day        240
morning    253
night      265
Name: daytime, dtype: int64

In [192]:
log['hour'] = log.time.dt.hour